run with tensorlab env

In [1]:
import numpy as np
import tensorflow as tf
import tflearn
import matplotlib.pyplot as plt
from tflearn.data_utils import shuffle
from numpy import genfromtxt


%matplotlib inline

In [92]:
merchants_tpvs = genfromtxt('merchants_tpvs3.csv', delimiter=',')

In [93]:
# for i in merchants_tpvs:
#     print (len(i))

In [94]:
len(merchants_tpvs)

100

In [95]:
shuffle = merchants_tpvs
np.random.shuffle(shuffle)
shuffle

array([[   9400.,       0.,       0., ...,   22290.,   89620.,   34900.],
       [      0.,       0.,       0., ...,   13450.,   56565.,   29240.],
       [      0.,       0.,       0., ...,       0.,       0.,       0.],
       ..., 
       [  14610.,       0.,   13250., ...,   19975.,   10990.,    6750.],
       [   9800.,       0.,    9900., ...,       0.,       0.,       0.],
       [  43157.,  110380.,   69259., ...,       0.,       0.,       0.]])

In [96]:
test_fraction = 0.9
records = len(merchants_tpvs)

train_X, test_X = shuffle[:int(records*test_fraction)], shuffle[int(records*test_fraction):]

In [97]:
train_X[6]

array([  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         1.15000000e+03,   3.23500000e+04,   1.96900000e+04,
         3.02700000e+04,   0.00000000e+00,   2.60700000e+04,
         1.77500000e+04,   6.34000000e+03,   8.38900000e+04,
         1.07600000e+04,   5.93000000e+03,   1.00000000e+02,
         6.07000000e+04,   2.30700000e+04,   1.18100000e+05,
         7.73500000e+04,   1.03570000e+05,   2.92400000e+04,
         2.04900000e+04,   6.55600000e+04,   2.80600000e+04,
         1.11500000e+04,   6.80000000e+03,   1.95400000e+04,
         4.58900000e+04,   0.00000000e+00,   1.99000000e+04,
         6.58200000e+04,   6.54000000e+03,   1.56000000e+04,
         7.65000000e+04,   2.29000000e+03,   2.60000000e+03,
         1.02970000e+05,   3.75600000e+04,   0.00000000e+00,
         3.24000000e+04,   1.39956000e+05,   2.17000000e+04,
         1.34000000e+03,   8.98800000e+04,   1.58000000e+04,
         9.16600000e+04,   1.53500000e+04,   9.00000000e+03,
         1.94700000e+04,

In [98]:
train_Y = np.array(train_X[0][244])
for i in train_X[1:]:
    train_Y = np.append( train_Y, i[244])
train_Y

array([  3.49000000e+04,   2.92400000e+04,   0.00000000e+00,
         1.23500000e+04,   0.00000000e+00,   4.46400000e+04,
         1.43000000e+04,   7.43500000e+03,   0.00000000e+00,
         2.00000000e+03,   1.70872000e+05,   3.84870000e+04,
         0.00000000e+00,   2.15800000e+04,   7.00320000e+04,
         1.92220000e+04,   0.00000000e+00,   1.92700000e+04,
         1.16413000e+05,   3.13900000e+04,   7.00000000e+03,
         5.23500000e+04,   1.34000000e+04,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         8.60000000e+03,   3.62417000e+05,   1.92600000e+04,
         0.00000000e+00,   0.00000000e+00,   2.18000000e+04,
         0.00000000e+00,   0.00000000e+00,   9.61000000e+04,
         3.23290000e+04,   0.00000000e+00,   0.00000000e+00,
         5.73590000e+04,   0.00000000e+00,   0.00000000e+00,
         6.93800000e+04,   8.10000000e+03,   5.20000000e+03,
         6.02500000e+03,   6.47900000e+04,   1.64010000e+05,
         3.84300000e+04,

In [99]:
test_Y = np.array(test_X[0][244])
for i in test_X[1:]:
    test_Y = np.append( test_Y, i[244])

In [100]:
new_train_X = np.array(train_X[0][:-1])
for i in train_X[1:]:
    new_train_X = np.append( new_train_X, i[:-1])


In [101]:
new_test_X = np.array(test_X[0][:-1])
for i in test_X[1:]:
    new_test_X = np.append( new_test_X, i[:-1])

In [102]:
new_test_X[0]

41580.0

In [103]:
new_train_X = np.reshape(new_train_X, (90, 244,1))/1000
train_Y = np.reshape(train_Y, (90,1))/1000
new_test_X = np.reshape(new_test_X, (10, 244,1))/1000
test_Y = np.reshape(test_Y, (10,1))/1000

In [109]:
# Define the neural network
def build_model():
    # This resets all parameters and variables, leave this here
    tf.reset_default_graph()

    # input layer
    net = tflearn.input_data(shape=[None, 244,1])
    #net = tflearn.embedding(net, input_dim=1, output_dim=60)
    # net = tflearn.lstm(net, n_units=30, return_seq=True)
    net = tflearn.lstm(net, n_units=256, return_seq=False)
    net = tflearn.fully_connected(net, 128, activation='linear')
    net = tflearn.fully_connected(net, 64, activation='linear')
    net = tflearn.fully_connected(net, 32, activation='linear')
    net = tflearn.fully_connected(net, 1, activation='linear')
    
    #Output Layer
    adam = tflearn.optimizers.Adam(learning_rate=0.0001, beta1=0.99)
    net = tflearn.regression(net, optimizer= adam, loss='mean_square')
    
    # This model assumes that your network is named "net"  
    #model = tflearn.DNN(net, clip_gradients=0.0, tensorboard_verbose=0)
    model = tflearn.DNN(net, clip_gradients=5.0)
    #model = tflearn.DNN(net)
    return model

In [110]:
model = build_model()


In [ ]:
# Training
model.fit(new_train_X, train_Y, n_epoch=50, validation_set=0.1, show_metric=True, batch_size=10, shuffle=True)
#model.fit(train_features, train_next_point, n_epoch=1, validation_set=0.1, show_metric=True, batch_size=1)

Training Step: 5  | total loss: 5479.08984 | time: 9.474s
| Adam | epoch: 001 | loss: 5479.08984 - binary_acc: 0.0050 -- iter: 50/81


In [71]:
# Compare the labels that our model predicts with the actual labels

# Find the indices of the most confident prediction for each item. That tells us the predicted digit for that sample.
predictions = np.array(model.predict(new_test_X))

# Calculate the accuracy, which is the percentage of times the predicated labels matched the actual labels
actual = test_Y
test_accuracy = np.mean(np.abs(predictions - actual), axis=0)

# Print out the result
print("Test accuracy: ", test_accuracy)

Test accuracy:  [ 284.13255063]


In [72]:
predictions

array([[  1.91128522e-03],
       [  7.19409436e-05],
       [  2.36255489e-03],
       [  2.34573148e-03],
       [  2.04339111e-03],
       [  2.10057851e-03],
       [  7.12962792e-05],
       [  7.13083209e-05],
       [  1.84747053e-03],
       [  2.09719781e-03]], dtype=float32)

In [73]:
actual

array([[   48.  ],
       [    0.  ],
       [   25.14],
       [  379.6 ],
       [   26.  ],
       [  540.  ],
       [    0.  ],
       [    0.  ],
       [ 1686.6 ],
       [  136.  ]])